In [1]:
import numpy as np
import gurobipy as gp

# Questions 1
Max is in a pie eating contest that lasts 1 hour. Each torte that he eats takes 2 minutes to eat. Each apple pie that he eats takes 3 minutes. He receives 4 points for each torte and 5 points for each pie. Find the number of tortes and apple pies Max should eat to get the most points. Solve the problem using the graphical method.

In [26]:
obj = np.array([4,5]) # want to maximize
A = np.zeros((1,2)) # initialize constraint matrix
A[0,:] = [2,3] 
b = np.array([60]) 
sense = np.array(['<']) 
print(A)

In [28]:
ojModel = gp.Model() # initialize an empty model

ojModX = ojModel.addMVar(2) # tell the model how many variables there are, where optimal objectives are stored
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) # add the constraints to the model (A, )
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

ojModel.Params.OutputFlag = 0 
ojModel.optimize()

In [29]:
pt_lover = ojModel.objVal # optimal revenue level

In [30]:
ojModX.x # tarts eaten vs pies eaten

array([30.,  0.])

### Question 1.b

Next, let’s see what happens if he would like to stick to his preference of eating at least as many pies as tortes. That is; the number of pies he eats should be greater than or equal to the number of tortes.

In [50]:
obj = np.array([4,5]) 
A = np.zeros((2,2)) 
A[0,:] = [2,3] 
A[1,:] = [-1,1] 
b = np.array([60,0]) 
sense = np.array(['<','>']) 
print(A)

[[ 2.  3.]
 [-1.  1.]]


In [21]:
ojModel = gp.Model() 
ojModX = ojModel.addMVar(2) 
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

ojModel.Params.OutputFlag = 0 
ojModel.optimize() 

In [25]:
pie_lover = ojModel.objVal # optimal revenue level

In [23]:
ojModX.x # tarts eaten vs pies eaten

### Question 1.c
By how many points does this constraint decrease Max’s total points? (this is the part you will submit to the canvas answer.)

In [31]:
pt_lover - pie_lover

12.0

### Answer: 
Max will earn 12 fewer points if he decides to eat at least as many pies as tortes.

# Question 2
A farmer in Iowa owns 450 acres of land. He is going to plant each acre with wheat or corn. Each acre planted with wheat (corn) yields $2,000 ($3,000) profit, requires three (two) workers, and requires two (four) tons of fertilizer. There are currently 1,000 workers and 1,200 tons of fertilizer available.

 

Formulate and solve this problem using gurobi.

 

In [32]:
obj = np.array([2000,3000]) 
A = np.zeros((3,2)) 
A[0,:] = [1,1] 
A[1,:] = [3,2] 
A[2,:] = [2,4] 
b = np.array([450,1000,1200]) 
sense = np.array(['<','<','<']) 

In [33]:
ojModel = gp.Model() 
ojModX = ojModel.addMVar(2) 
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 

ojModel.Params.OutputFlag = 0 
ojModel.optimize()

In [34]:
ojModel.objVal

1000000.0

In [35]:
ojModX.x

array([200., 200.])

With these contstraints, the framer's revenue is $1,000,000. This is the with him planting wheat on 200 acres and corn on 200 acres and leaving 50 acres unplanted.

### Question 2.b
Next we want to see What happens to the decision variables and the total profit when the availability of fertilizer varies from 200 tons to 2200 tons in 100-ton increments.

 

At what level of fertilizer does the farmer discontinue producing wheat?  That is, what is the smallest level of available fertilizer that results in no wheat being produced?  Your answer should be one of the 100 ton incremental numbers, like 600 or 1300 or...

In [49]:
i = 200
rev= []
num = []
while i <= 2200:
    obj = np.array([2000,3000]) 
    A = np.zeros((3,2)) 
    A[0,:] = [1,1] 
    A[1,:] = [3,2]
    A[2,:] = [2,4] 
    b = np.array([450,1000,i])
    sense = np.array(['<','<','<'])
    
    ojModel = gp.Model() 
    ojModX = ojModel.addMVar(2) 
    
    ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
    ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 
    ojModel.Params.OutputFlag = 0 
    ojModel.optimize()
    rev = ojModel.objVal
    num = ojModX.x
    print ('{0} tons of fertilizer produces {1} acres of wheat and corn'.format(i,num))
    i = i + 100

200 tons of fertilizer produces [100.   0.] acres of wheat and corn
300 tons of fertilizer produces [150.   0.] acres of wheat and corn
400 tons of fertilizer produces [200.   0.] acres of wheat and corn
500 tons of fertilizer produces [250.   0.] acres of wheat and corn
600 tons of fertilizer produces [300.   0.] acres of wheat and corn
700 tons of fertilizer produces [325.   12.5] acres of wheat and corn
800 tons of fertilizer produces [300.  50.] acres of wheat and corn
900 tons of fertilizer produces [275.   87.5] acres of wheat and corn
1000 tons of fertilizer produces [250. 125.] acres of wheat and corn
1100 tons of fertilizer produces [225.  162.5] acres of wheat and corn
1200 tons of fertilizer produces [200. 200.] acres of wheat and corn
1300 tons of fertilizer produces [175.  237.5] acres of wheat and corn
1400 tons of fertilizer produces [150. 275.] acres of wheat and corn
1500 tons of fertilizer produces [125.  312.5] acres of wheat and corn
1600 tons of fertilizer produces

### Answer: 
1800 tons of fertlizer is smallest about of ferilizer that produces no wheat

# Question 3
Star Oil Company is considering five different investment opportunities. The table below gives the required cash outflows and net present values in millions of dollars.

Star Oil has $40 million available for investment now (time 0); it estimates that one year from now (time 1) $20 million will be available for investment. Star Oil may purchase any fraction of each investment, but no more than 100% of each opportunity. In this case, the cash outflows and NPV are adjusted accordingly.

For example, if Star Oil purchases one-fifth of investment 3, then a cash outflow of 1/5 × 5 = 1 million dollars would be required at time 0, and a cash outflow of 1/5 × 5 = 1 million would be required at time 1. The one-fifth share of investment three would yield an NPV of 1/5 ∗ 16 = 3.2 million dollars. Star Oil wants to maximize the NPV that can be obtained by investing in investments 1-5. Formulate an LP that will help achieve this goal. Assume that any funds leftover at time 0 cannot be used at time 1.

 
What percentage of opportunity 3 should be Star Oil invest in?  Answer in decimals, so if your answer is 54%, you should input 0.54.  Round 2 to decimal places

In [51]:
obj = np.array([13,16,16,14,39]) 
A = np.zeros((7,5)) 
A[0,:] = [11,53,5,5,29] 
A[1,:] = [3,6,5,1,34] 
A[2,:] = [1,0,0,0,0] 
A[3,:] = [0,1,0,0,0] 
A[4,:] = [0,0,1,0,0] 
A[5,:] = [0,0,0,1,0] 
A[6,:] = [0,0,0,0,1] 
b = np.array([40,20,1,1,1,1,1]) 
sense = np.array(['<','<','<','<','<','<','<'])
print(A)

[[11. 53.  5.  5. 29.]
 [ 3.  6.  5.  1. 34.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]


In [52]:
ojModel = gp.Model() 
ojModX = ojModel.addMVar(5) 
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 
ojModel.Params.OutputFlag = 0 
ojModel.optimize()

In [53]:
ojModel.objVal

57.449017199017206

In [56]:
v = ojModX.x
v

array([1.        , 0.20085995, 1.        , 1.        , 0.28808354])

In [57]:
v*[11,53,5,5,29]

array([11.       , 10.6455774,  5.       ,  5.       ,  8.3544226])

### Answer: 
Star Oil should invest 100% in oppurtunity 3.

# Question 4
The goal of this problem is to select a set of foods that will satisfy a set of daily nutritional requirement at minimum cost. Suppose there are three foods available, corn, milk, and bread. There are restrictions on the number of calories (between 2000 and 2250) and the amount of Vitamin A (between 5000 and 50,000) that can be eaten. The table below shows, for each food, the cost per serving, the amount of Vitamin A per serving, and the number of calories per serving. Also, the maximum number of servings for each food is 10.

How many servings of corn should you eat?  Round to 2 decimal places.

In [7]:
obj = np.array([.18,.23,.05]) 
A = np.zeros((7,3)) 
A[0,:] = [72,121,65] 
A[1,:] = [72,121,65] 
A[2,:] = [107,500,0] 
A[3,:] = [107,500,0] 
A[4,:] = [1,0,0] 
A[5,:] = [0,1,0] 
A[6,:] = [0,0,1] 

b = np.array([2000,2250,5000,50000,10,10,10]) 
sense = np.array(['>','<','>','<','<','<','<']) 
print(A)

[[ 72. 121.  65.]
 [ 72. 121.  65.]
 [107. 500.   0.]
 [107. 500.   0.]
 [  1.   0.   0.]
 [  0.   1.   0.]
 [  0.   0.   1.]]


In [11]:
ojModel = gp.Model() 
ojModX = ojModel.addMVar(3) 
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE) 

ojModel.Params.OutputFlag = 0 
ojModel.optimize()

In [12]:
ojModel.objVal

3.1500000000000004

In [13]:
ojModX.x

array([ 1.94444444, 10.        , 10.        ])

### Answer: 
You should eat 1.94 servings of corn.

# Question 5
Paper and wood products companies need to define cutting schedules that will maximize the total wood yield of their forests over some planning period. Suppose that a firm with control of 2 forest units wants to identify the best cutting schedule over a planning horizon of 3 years. Forest unit 1 has a total acreage of 2 and unit 2 has a total of 3 acres. The studies that the company has undertaken predict that each acre in unit 1(2) will have 1, 1.3, 1.4 (1, 1.2, 1.6) tons of woods per acre available for harvesting in year 1, 2, 3 respectively. Based on its prediction of economic conditions, the company believes that it should harvest at least 1.2, 1.5, 2 tons of wood in year 1, 2, 3 separately. Due to the availability of equipment and personnel, the company can harvest at most 2, 2, 3 tons of wood in year 1, 2, 3. Find the company’s best cutting strategy that maximizes the total weights of wood. Here discounting of the time value should not be considered.  If some fraction of a forest unit is cut down in year 1, that part of the forest cannot be cut again for the remaining 2 years.  Similarly if some fraction of the forest unit is cut down in year 2 it cannot be cut in year 3.

 

In year 3, how many acres of forest unit 2 should be cut down?  Round to 2 decimal places.

In [17]:
obj = np.array([1,1,1,1,1,1]) 
A = np.zeros((8,6)) 
A[0,:] = [1,1,1,0,0,0] 
A[1,:] = [0,0,0,1,1,1] 
A[2,:] = [2,0,0,3,0,0] 
A[3,:] = [2,0,0,3,0,0]
A[4,:] = [0,2.6,0,0,3.6,0]
A[5,:] = [0,2.6,0,0,3.6,0]
A[6,:] = [0,0,2.8,0,0,4.8]
A[7,:] = [0,0,2.8,0,0,4.8]


b = np.array([1, 1, 1.2, 2, 1.5, 2, 2, 3]) 
sense = np.array(['<','<','>','<','>','<','>','<'])
print(A)

[[1.  1.  1.  0.  0.  0. ]
 [0.  0.  0.  1.  1.  1. ]
 [2.  0.  0.  3.  0.  0. ]
 [2.  0.  0.  3.  0.  0. ]
 [0.  2.6 0.  0.  3.6 0. ]
 [0.  2.6 0.  0.  3.6 0. ]
 [0.  0.  2.8 0.  0.  4.8]
 [0.  0.  2.8 0.  0.  4.8]]


In [18]:
ojModel = gp.Model() 
ojModX = ojModel.addMVar(6) 
    
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 
ojModel.Params.OutputFlag = 0 
ojModel.optimize()

In [20]:
ojModel.x

[0.23076923076923084, 0.7692307692307692, 0.0, 0.375, 0.0, 0.625]

In [21]:
0.625*3

1.875

### Answer:
In year 3, forest 2, they should cut down 1.875 (1.88) acres.

0.625 is the percent of forest unit 2 that should be cut down then, and 3 is how many acres are in that unit.